In [ ]:
import cv2
import IPython.display 
import numpy as np
import os
from skimage.external.tifffile import imread
import render_gt
import torch

def imshow(img):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)
    
def showCube(vol):
    v1=np.max(vol,axis=0)
    v2=np.max(vol,axis=1)
    v3=np.max(vol,axis=2)
    imshow(v1*255)
    imshow(v2*255)
    imshow(v3*255)

In [ ]:
# render AIY

imgDir="AIY_neuron/AIY_tif/"
swcDir="AIY_neuron/AIY_swc/"
lblDir="AIY_neuron/AIY_gt/"
os.rename("AIY_neuron/AIY_swc/Worm_ADS3(10)_red_t100_AIY_part.swc",
          "AIY_neuron/AIY_swc/Worm_ADS3(10)_red_t100_part_AIY.swc")
os.mkdir(lblDir)

m=[1,1,1]
o=[0,0,0]
scale_factor=[1,1,1]
downsampling=torch.tensor(scale_factor,dtype=torch.double)
offset=torch.tensor(o,dtype=torch.double)
scale=torch.tensor(m,dtype=torch.double)
volDims=torch.tensor([4,3,2],dtype=torch.long)

for fname in os.listdir(imgDir):
    print(fname)
    bname,ext=os.path.splitext(fname)
    #print(bname,ext)
    if ext==".tif":
        swcname=os.path.join(swcDir,bname+"_AIY.swc")
        if os.path.isfile(swcname):
            img=imread(os.path.join(imgDir,fname)).astype(np.float)
            print(img.max())
            img=img/img.max()
            vol=np.zeros(img.shape,dtype=np.uint8)
            render_gt.renderSWC2volume(swcname, volDims, vol, scale, offset, downsampling)
            showCube((img+vol)/2)
            np.save(os.path.join(lblDir,bname),vol)
        else:
            print("file {} does not exist".format(swcname))
    else:
        print("encoured unexpected extension: {}".format(ext))



In [ ]:
# view rendered AIY

imgDir="AIY_neuron/AIY_tif/"
lblDir="AIY_neuron/AIY_gt/"
for fname in os.listdir(imgDir):
    print(os.path.join(imgDir,fname))
    bname,ext=os.path.splitext(fname)
    if ext==".tif":
        img=imread(os.path.join(imgDir,fname)).astype(np.float)
        lfile=os.path.join(lblDir,bname+".npy")
        if os.path.isfile(lfile):
            lbl=np.load(lfile).astype(np.float)
            print(img.max())
            img=img/img.max()
            showCube(img)
            showCube(lbl)
        else:
            print("file {} does not exist".format(lfile))

In [ ]:
# render AIA and AIB

imgDir="AIA&AIB_neurons/AIA&AIB_tif"
swcDirs=["AIA&AIB_neurons/AIA_swc/","AIA&AIB_neurons/AIB_swc/"]
suffixes=["AIA","AIB"]
lblDir="AIA&AIB_neurons/gt/"
os.mkdir(lblDir)

m=[1,1,1]
o=[0,0,0]
scale_factor=[1,1,1]
downsampling=torch.tensor(scale_factor,dtype=torch.double)
offset=torch.tensor(o,dtype=torch.double)
scale=torch.tensor(m,dtype=torch.double)
volDims=torch.tensor([4,3,2],dtype=torch.long)

for fname in os.listdir(imgDir):
    print(fname)
    bname,ext=os.path.splitext(fname)
    #print(bname,ext)
    if ext==".tif":
        img=imread(os.path.join(imgDir,fname)).astype(np.float)
        print(img.max())
        #img=img/(255.0*255.0)
        img=img/img.max()
        vol=np.zeros(img.shape,dtype=np.uint8)
        for swcDir,suffix in zip(swcDirs,suffixes):
            swcname=os.path.join(swcDir,bname+"_"+suffix+".swc")
            print(swcname)
            if os.path.isfile(swcname):
                render_gt.renderSWC2volume(swcname, volDims, vol, scale, offset, downsampling)
            else:
                print("file {} does not exist".format(swcname))
        showCube((img+vol)/2)
        np.save(os.path.join(lblDir,bname),vol)
    else:
        print("encountered unexpected extension: {}".format(ext))



In [ ]:
# view rendered AIA and AIB
imgDir="AIA&AIB_neurons/AIA&AIB_tif"
lblDir="AIA&AIB_neurons/gt/"
for fname in os.listdir(imgDir):
    print(os.path.join(imgDir,fname))
    bname,ext=os.path.splitext(fname)
    if ext==".tif":
        img=imread(os.path.join(imgDir,fname)).astype(np.float)
        lbl=np.load(os.path.join(lblDir,bname+".npy")).astype(np.float)
        print(img.max())
        #img=img/(255.0*255.0)
        img=img/img.max()
        showCube(img)
        showCube(lbl)
    